In [ ]:
import cv2
import numpy as np

img = cv2.imread('sign.png')
print
white_mask = cv2.inRange(img, np.array([200, 200, 200]), np.array([255, 255, 255]))
result = cv2.cvtColor(img, cv2.COLOR_BGR2BGRA)

result[white_mask > 0] = [0, 0, 0, 0]

cv2.imshow('Original', img)
cv2.imshow('Background Removed', result)

cv2.imwrite('result.png', result)

cv2.waitKey()
cv2.destroyAllWindows()


In [ ]:
import cv2

img = cv2.imread('id-card.jpg')
scale_percent = 50 
width = int(img.shape[1] * scale_percent / 100)
height = int(img.shape[0] * scale_percent / 100)
dim = (width, height)
resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
# cv2.imwrite('resized_id_card.jpg', resized)

def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        original_x = int(x / scale_percent * 100)
        original_y = int(y / scale_percent * 100)
        print(f"Clicked at (resized): ({x}, {y})")

        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(resized, f"({x},{y})", (x, y), font, 0.5, (255, 0, 0), 2)
        cv2.imshow('image', resized)

cv2.imshow('image', resized)
cv2.setMouseCallback('image', click_event)
cv2.waitKey(0)
cv2.destroyAllWindows()


Clicked at (resized): (538, 369)
Clicked at (resized): (688, 435)


In [ ]:

import cv2
import numpy as np

image = cv2.imread('resized_id_card.jpg')
mask = np.zeros(image.shape[:2], dtype=np.uint8)

# Region to inpaint (previous signature region)
cv2.rectangle(mask, (540, 366), (660, 437), 255, -1)
inpainted_image = cv2.inpaint(image, mask, inpaintRadius=3, flags=cv2.INPAINT_TELEA)

cv2.imshow('Original Image', image)
cv2.imshow('Mask', mask)
cv2.imshow('Inpainted Image', inpainted_image)
cv2.imwrite('new_img.png', inpainted_image)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
import cv2
import numpy as np

x_pos, y_pos = 500, 400

background = cv2.imread('new_img.png')
overlay = cv2.imread('result.png', cv2.IMREAD_UNCHANGED)

b, g, r, a = cv2.split(overlay)
alpha_factor = 0.95  
a = (a * alpha_factor).astype(np.uint8)
overlay = cv2.merge([b, g, r, a])

bg_h, bg_w = background.shape[:2]

scale_factor = 2.0  
new_w = int(overlay.shape[1] * scale_factor)
new_h = int(overlay.shape[0] * scale_factor)

new_w = min(new_w, bg_w - x_pos)
new_h = min(new_h, bg_h - y_pos)

overlay_resized = cv2.resize(overlay, (new_w, new_h), interpolation=cv2.INTER_AREA)

for y in range(new_h):
    for x in range(new_w):
        if overlay_resized[y, x, 3] > 0:
            alpha = overlay_resized[y, x, 3] / 255
            bg_color = background[y + y_pos, x + x_pos]
            overlay_color = overlay_resized[y, x, :3]
            background[y + y_pos, x + x_pos] = bg_color * (1 - alpha) + overlay_color * alpha


cv2.imshow('Combined Image', background)
cv2.imwrite('final_id_sign.jpg', background)
cv2.waitKey(0)
cv2.destroyAllWindows()
